# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
import sys
from sklearn.decomposition import NMF, LatentDirichletAllocation
import matplotlib.pyplot as plt

In [ ]:
!pip install stop_words
from stop_words import get_stop_words

stop_words = get_stop_words('english')
stop_words.append('said')
all_words = stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp37-none-any.whl size=32917 sha256=7191a36099f9e6e079b1d1232827555d6491b18da694237846e50a328a3440e9
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [ ]:
documents=[]
for year in range(1981,2020):
    i=0
    file = pd.read_csv(f'/content/drive/MyDrive/articler_data/new_article_{year}.csv')
    file = file.dropna().reset_index().drop('index',axis=1)
#     file = file['article']
#     file = file.dropna()
    docs = []
    for j in range(0, len(file)):
        documents.append(file.loc[j,'article_cleaned_lemmatizer'])
    print(year,end=', ')
#     documents.append(docs)

1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 

In [ ]:
n_features = 1000
n_components = [5,10,15,20,25,30,40,50]
n_top_words = 20

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words=all_words)
tf = tf_vectorizer.fit_transform(documents)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aren', 'can', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
for n in n_components:
  lda = LatentDirichletAllocation(n_components=n, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

  lda.fit(tf)
  distribution = lda.transform(tf)
  dist = pd.DataFrame(distribution)
  dist.to_csv(f'dist_{n}.csv',index=False)
  print(n, 'done')

40 done
50 done


In [ ]:
n_topics = [5,10,15,20,25,30,40,50]
no_years = 2020 - 1981

for no in n_topics:
  distribution = pd.read_csv(f'/content/dist_{no}.csv')
  weights = np.zeros((no_years, no))
  start = 0
  for i in range(1981, 2020):
    file = pd.read_csv(f'/content/drive/MyDrive/articler_data/new_article_{i}.csv')
    year = file.dropna()
		year_data = distribution[start:start+len(year),:]
    yearly_sum = np.sum(year_data, axis = 0)/ len(year)
		weights[i - 1981] = yearly_sum
		start = start + len(year)
  wts_df = pd.DataFrame(weights)
  wts_df.to_csv(f'/content/drive/MyDrive/weights/weights_{no}.csv',index=False)
  print(no, 'done')

TabError: ignored